In [2]:
import ftplib
from ftplib import FTP
from io import StringIO
import os
from pathlib import Path
import site
import sys

In [3]:
BASEDIR = Path.home() / 'Development/hw-4.2.0'
site.addsitedir(str(BASEDIR / 'hw'))

In [4]:
from tools import *
from program import Program

In [5]:
BASEDIR = Path.home()

In [6]:
cd(BASEDIR)

In [7]:
conf = configure()['ENVIRONMENT']
[item for item in conf.items()]

[('server_ip', '192.168.254.71'),
 ('ftp_pass', '.config/hw/ftp.cnf'),
 ('program', 'hw'),
 ('version', '4.2.0')]

## Connect to the FTP Server

In [8]:
def login():
    password = (Path.home() / conf['ftp_pass']).read_text()
    ftp = FTP(conf['server_ip'])
    ftp.login('russell', password)
    return ftp

## Change Remote Directory

In [11]:
ftp = login()
ftp.cwd('/srv/www/htdocs/library/books/test')

'250 Directory successfully changed.'

In [46]:
ftp.retrlines('PUT {

'250'

## List the Current Directory

In [38]:
sys.stdout = StringIO()
ftp.retrlines('LIST')

'226 Directory send OK.'

drwxr-xr-x    1 1000     100             0 Aug 17 03:57 books



In [39]:
s = sys.stdout.getvalue()
sys.stdout = stdout
print(s)

In [12]:
help(ftp.putline)

Help on method putline in module ftplib:

putline(line) method of ftplib.FTP instance
    # Internal: send one line to the server, appending CRLF



In [40]:
print('hi')

hi


## Determine if a Remote File Exists

In [155]:
def exists(ftp, s):
    cur_dir = ftp.pwd()
    try:
        ftp.cwd(s)
        ftp.cwd(cur_dir)
        return True
    except ftplib.error_perm as e:
        print(e)
        lastresp = ftp.lastresp
        ftp.cwd(cur_dir)
        try:
            ftp.size(s)
            # ftp.cwd(cur_dir)
            return True
        except ftplib.error_perm as e:
            print(e)
            # ftp.cwd(cur_dir)
            return False
        # return False
    ftp.cwd(cur_dir)

exists(ftp, '/etc')

True

In [156]:
def make_remote_dirs(ftp, s):
    components = s.split(os.sep)
    i = -1
    pp(components)
    s0 = os.sep.join(components[:i])
    print(s0)
    while len(s0) and not exists(ftp, s0):
         print(f'Directory {s0} does not exist.')
         i -= 1
         s0 = os.sep.join(components[:i])
         print(f'Next directory to check: {s0}')
    print(len(s0))
    while i < 0:
        i += 1
        s0 = os.sep.join(components[:i])
        if len(s0):
            print(f'Making directory {s0}')
            ftp.mkd(s0)

In [157]:
# login()
make_remote_dirs(ftp, '600/620/628/Boring Stuff')

['600', '620', '628', 'Boring Stuff']
600/620/628
550 Failed to change directory.
550 Could not get file size.
Directory 600/620/628 does not exist.
Next directory to check: 600/620
550 Failed to change directory.
550 Could not get file size.
Directory 600/620 does not exist.
Next directory to check: 600
550 Failed to change directory.
550 Could not get file size.
Directory 600 does not exist.
Next directory to check: 
0
Making directory 600
Making directory 600/620
Making directory 600/620/628


In [130]:
public(str)

capitalize  format      isidentifier  ljust      rjust       swapcase 
casefold    format_map  islower       lower      rpartition  title    
center      index       isnumeric     lstrip     rsplit      translate
count       isalnum     isprintable   maketrans  rstrip      upper    
encode      isalpha     isspace       partition  split       zfill    
endswith    isascii     istitle       replace    splitlines
expandtabs  isdecimal   isupper       rfind      startswith
find        isdigit     join          rindex     strip     


In [138]:
ftp.pwd()

'/srv/www/htdocs/library/books/test'

In [112]:
ftp.size('books/3 Little Pigs')

0